In [ ]:
!pip install Faker
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.6/379.6 KB 2.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for psycopg2, since package 'wheel' is not installed.
  Running setup.py install for psycopg2 ... done


In [2]:
import random
from faker import Faker
from datetime import datetime, timedelta
import uuid

In [3]:
def generate_sample_data():
    fake = Faker("es_ES")

    # Generate 50 fake users
    users = []
    for i in range(0, 50):
        user_id = fake.uuid4()
        users.append(
            {
                "user_id": user_id,
                "nombre": fake.name(),
                "email": fake.email(),
                "telefono": fake.phone_number(),
                "direccion": fake.address(),
                "fecha_registro": (
                    datetime.now() - timedelta(days=random.randint(180, 365))
                ).strftime("%Y-%m-%d %H:%M:%S"),
            }
        )
    print("--- 50 Sample Users (Salvadoran Spanish) ---")
    print(users[:10])

    # Generate 6 months of billing data for each user
    billing_data = []
    service = "Internet Residencial"
    base_amount = 35.00
    today = datetime.now()

    for user in users:
        for month_offset in range(6, 0, -1):
            bill_date = today - timedelta(days=30 * month_offset)
            amount = base_amount + round(random.uniform(-5.00, 5.00), 2)
            status = random.choice(["Cancelado", "Pendiente"])

            billing_id = fake.uuid4()
            billing_data.append(
                {
                    "billing_id": billing_id,
                    "user_id": user["user_id"],
                    "fecha_factura": bill_date.strftime("%Y-%m-%d 00:00:00"),
                    "monto": amount,
                    "estado": status,
                    "servicio": service,
                }
            )
    print("\n--- 6 Months of Billing Data for 'Internet Residencial' ---")
    print(billing_data[:10])
    return users, billing_data

In [4]:
users, billing_data = generate_sample_data()

--- 50 Sample Users (Salvadoran Spanish) ---
[{'user_id': 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', 'nombre': 'Julieta Riba Borrego', 'email': 'juan-luis71@example.net', 'telefono': '+34732 823 807', 'direccion': 'Paseo Amaro Duque 871\nAlmería, 46256', 'fecha_registro': '2024-12-29 00:25:14'}, {'user_id': 'bb81a4eb-c7ed-4411-93a7-01f250c6a812', 'nombre': 'Angelita de Salinas', 'email': 'ariascristobal@example.org', 'telefono': '+34810454551', 'direccion': 'Vial de Fidela Mármol 93 Puerta 2 \nÁlava, 44651', 'fecha_registro': '2025-02-22 00:25:14'}, {'user_id': 'd61e9531-e34d-4aa9-beeb-ab59f84c093e', 'nombre': 'Custodia Pedro Batlle', 'email': 'eliasapolonia@example.net', 'telefono': '+34844901334', 'direccion': 'Pasadizo de Eugenio Doménech 82 Puerta 1 \nLeón, 37097', 'fecha_registro': '2025-02-20 00:25:14'}, {'user_id': 'deafc729-f17e-4864-a742-db5593c57a47', 'nombre': 'Rosario Zaragoza Cal', 'email': 'sandaliomarino@example.com', 'telefono': '+34878442963', 'direccion': 'Glorieta de Al

In [5]:
# Parámetros de conexión MYSQL
config = {
    "user": "demo",
    "password": "demo",
    "host": "127.0.0.1",
    "database": "icc115",
    'port': '5432'
}

In [14]:
import psycopg2
from psycopg2 import DatabaseError

def insert_multiple_records(records, sql_query):
    """Inserta múltiples registros en PostgreSQL utilizando psycopg2."""
    conn = None # Inicializar conn fuera del try para el bloque finally
    try:
        # Establecer la conexión con la base de datos
        conn = psycopg2.connect(**config)
        cursor = conn.cursor()

        # Ejecutar la inserción masiva
        # Note: psycopg2 usa '%s' como marcador de posición, no '%s' ni '?'
        cursor.executemany(sql_query, records)

        # Confirmar los cambios (obligatorio para guardar los datos)
        conn.commit()

        # psycopg2 no tiene rowcount tan fácilmente después de executemany,
        # pero podemos usar el del cursor.
        print(f"{cursor.rowcount} registros insertados correctamente.")

    except DatabaseError as e:
        print(f"Error de base de datos al insertar múltiples registros: {e}")
        if conn:
            conn.rollback() # Revertir cambios ante un error
        # Re-lanzar la excepción para que el código que llama sepa que falló
        # O se puede simplemente retornar
        # raise 
        
    except Exception as e:
        # Capturar otros errores como ConfigurationError, etc.
        print(f"Error inesperado: {e}")
        
    finally:
        # Cerrar el cursor y la conexión
        if conn:
            if cursor:
                cursor.close()
            conn.close()
            print("Conexión a PostgreSQL cerrada.")


def convertir_dict_a_tupla(lista_de_diccionarios, orden_columnas):
    """
    Convierte una lista de diccionarios a una lista de tuplas,
    garantizando el orden de los valores. (Esta función NO requiere cambios de driver)

    Args:
        lista_de_diccionarios (list): La lista de diccionarios a convertir.
        orden_columnas (list): Una lista de strings con las claves en el orden deseado.

    Returns:
        list: Una lista de tuplas con los valores ordenados.
    """
    records_as_tuples = []
    for record in lista_de_diccionarios:
        try:
            # Obtener los valores en el orden especificado
            tuple_record = tuple(record[key] for key in orden_columnas)
            records_as_tuples.append(tuple_record)
        except KeyError as e:
            print(
                f"Advertencia: El diccionario {record} no contiene la clave {e}. Se omite."
            )
    return records_as_tuples

In [15]:
# Insertando usuarios
insert_users_sql_query = "INSERT INTO users (user_id, nombre, email, telefono, direccion, fecha_registro) VALUES (%s, %s, %s, %s, %s, %s)"
tuplas_users = convertir_dict_a_tupla(
    users, ["user_id", "nombre", "email", "telefono", "direccion", "fecha_registro"]
)
print(tuplas_users[:10])
insert_multiple_records(tuplas_users, insert_users_sql_query)

[('ed3b8fb4-1999-4f05-b654-5b7df2aeef82', 'Julieta Riba Borrego', 'juan-luis71@example.net', '+34732 823 807', 'Paseo Amaro Duque 871\nAlmería, 46256', '2024-12-29 00:25:14'), ('bb81a4eb-c7ed-4411-93a7-01f250c6a812', 'Angelita de Salinas', 'ariascristobal@example.org', '+34810454551', 'Vial de Fidela Mármol 93 Puerta 2 \nÁlava, 44651', '2025-02-22 00:25:14'), ('d61e9531-e34d-4aa9-beeb-ab59f84c093e', 'Custodia Pedro Batlle', 'eliasapolonia@example.net', '+34844901334', 'Pasadizo de Eugenio Doménech 82 Puerta 1 \nLeón, 37097', '2025-02-20 00:25:14'), ('deafc729-f17e-4864-a742-db5593c57a47', 'Rosario Zaragoza Cal', 'sandaliomarino@example.com', '+34878442963', 'Glorieta de Alma Sebastián 40 Apt. 55 \nGirona, 24872', '2024-11-20 00:25:14'), ('b99d5157-b659-4c77-9ef9-65b7fd806d87', 'Baldomero Rius Cifuentes', 'cortesepifanio@example.org', '+34841 230 927', 'Calle Fabiana Torrents 715 Piso 3 \nLugo, 03854', '2025-01-09 00:25:14'), ('28fde1db-c7d5-4143-9783-92abcbb92672', 'Hernando de Losada'

In [19]:
# Insertando facturas
insert_billing_data_sql_query = "INSERT INTO billing_data (billing_id, user_id, fecha_factura, monto, estado, producto_id) VALUES (%s, %s, %s, %s, %s, %s)"

billing_data_updated = []
productos = [
    "c9a0b3e1-ee0c-44d0-accb-959b75cec5ad",
    "d5c151fd-0c02-49c3-858a-ccd80a870a76",
    "d5c151fd-0c02-49c3-858a-ccd80a870a76",
]
for record in billing_data:
    record["producto_id"] = productos[random.randint(0, 2)]
    billing_data_updated.append(record)
tuplas_billing_data = convertir_dict_a_tupla(
    billing_data,
    ["billing_id", "user_id", "fecha_factura", "monto", "estado", "producto_id"],
)
print(tuplas_billing_data[:10])
insert_multiple_records(tuplas_billing_data, insert_billing_data_sql_query)

[('5be205c9-747b-4120-b468-d485c21ffe4d', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', '2025-04-25 00:00:00', 30.78, 'Pendiente', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('3826c4e5-9cd3-4dcb-b569-48db6ac699f4', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', '2025-05-25 00:00:00', 35.24, 'Cancelado', 'c9a0b3e1-ee0c-44d0-accb-959b75cec5ad'), ('f656b1b5-12a4-45e9-ad9a-26c6ab3aee6a', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', '2025-06-24 00:00:00', 31.41, 'Cancelado', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('e4002af3-4b40-4dce-83b1-ae4d07d40a27', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', '2025-07-24 00:00:00', 36.18, 'Cancelado', 'c9a0b3e1-ee0c-44d0-accb-959b75cec5ad'), ('b9d2fb5f-c5fc-48c5-bc07-6dc0b6fa4d17', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', '2025-08-23 00:00:00', 38.43, 'Pendiente', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('52ce6487-cd2f-4bf8-9fd6-4fe8fa706f0c', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82', '2025-09-22 00:00:00', 35.83, 'Pendiente', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('bcfdf88

In [20]:
import json


def cargar_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            print("Datos cargados desde el archivo:")
            print(data[:1])
            print("El tipo de dato cargado es:", type(data))
            return data
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no fue encontrado.")
    except json.JSONDecodeError:
        print(
            "Error: No se pudo decodificar el JSON. Asegúrate de que el formato sea correcto."
        )

### Cargando conversaciones

In [21]:
conversations = []
new_client_sample_conversations_file_path = "./new_client_sample_conversations.json"
new_client_sample_conversations = cargar_json(new_client_sample_conversations_file_path)
conversations.extend(new_client_sample_conversations)

billing_sample_conversations_file_path = "./billing_sample_conversations.json"
billing_sample_conversations = cargar_json(billing_sample_conversations_file_path)
conversations.extend(billing_sample_conversations)

conexion_issues_conversations_file_path = "./conexion_issues_conversations.json"
conexion_issues_conversations = cargar_json(conexion_issues_conversations_file_path)
conversations.extend(conexion_issues_conversations)

print(conversations[:2])

Datos cargados desde el archivo:
[{'cliente': {'nombre_completo': 'Laura Patricia Vásquez Orellana', 'telefono': '7987-6543'}, 'satisfaccion': 5, 'historial': [{'sender': 'cliente', 'text': 'Hola, estoy interesada en contratar un plan de internet residencial. ¿Qué opciones tienen?'}, {'sender': 'agente', 'text': '¡Buenos días, Sra. Vásquez! Tenemos varios planes residenciales con diferentes velocidades. Para poderle dar la mejor recomendación, ¿podría indicarme su DUI para verificar la cobertura en su área?'}, {'sender': 'cliente', 'text': 'Sí, claro. Mi DUI es 09876543-2. Vivo en la Colonia Escalón.'}, {'sender': 'agente', 'text': 'Gracias. He verificado y su zona cuenta con cobertura de fibra óptica. Le recomiendo nuestro plan de 100 Mbps, que incluye una instalación gratuita por promoción. ¿Le gustaría que le brinde más detalles?'}, {'sender': 'cliente', 'text': 'Me parece muy bien. Sí, por favor, envíeme la información completa al correo electrónico.'}, {'sender': 'agente', 'text':

In [26]:
import psycopg2
def load_users_from_postgres():
    """
    Carga todos los registros de la tabla 'users' desde PostgreSQL.
    
    Args:
        config (dict): Diccionario de parámetros de conexión a PostgreSQL.
        
    Returns:
        list: Una lista de diccionarios (registros de usuarios) o None en caso de error.
    """
    conn = None
    try:
        # 1. Establecer la conexión con la base de datos PostgreSQL
        conn = psycopg2.connect(**config)
        
        # 2. El cursor por defecto de psycopg2 NO devuelve diccionarios.
        #    Usamos un módulo extra para obtener un cursor que devuelva diccionarios.
        from psycopg2.extras import RealDictCursor 
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        
        # 3. Ejecutar la consulta SQL (es casi idéntica, solo se eliminan las funciones MySQL)
        cursor.execute(
            "SELECT user_id, nombre, email, telefono, direccion, fecha_registro FROM users"
        )
        users_from_db = cursor.fetchall()
        
        print("Usuarios cargados desde PostgreSQL:")
        print(users_from_db[:5])
        return users_from_db
        
    except DatabaseError as e:
        # Manejo de errores específico de psycopg2
        print(f"Error de base de datos al cargar usuarios: {e}")
        return None
        
    except Exception as e:
        # Otros errores (configuración, etc.)
        print(f"Error inesperado al cargar usuarios: {e}")
        return None
        
    finally:
        # Cerrar el cursor y la conexión
        if conn:
            # No es necesario verificar conn.is_connected() con psycopg2 antes de close()
            cursor.close()
            conn.close()
            print("Conexión a PostgreSQL cerrada.")

In [27]:
import random

conversations_data = []
users_db = load_users_from_postgres()
for conversation in conversations:
    # Generar un número entero aleatorio entre 1 y 10 (inclusive)
    random_int = random.randint(0, 49)
    ramdom_user = users_db[random_int]
    session_id = str(uuid.uuid4())
    random_days_ago = random.randint(0, 180)
    random_date = datetime.now() - timedelta(days=random_days_ago)

    start_time = random_date  # Timestamp inicial de la conversación

    conversation_data = {}
    conversation_data["session_id"] = session_id
    conversation_data["user_id"] = ramdom_user["user_id"]
    # Array para almacenar los mensajes de la conversación
    conversation_mensajes = []

    # Iterar sobre los mensajes de la conversación
    for i, message in enumerate(conversation["historial"]):
        random_minutes = random.randint(0, 15)
        current_time = start_time + timedelta(minutes=random_minutes)
        message_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
        message_id = str(uuid.uuid4())

        message_data = {
            "session_id": session_id,
            "message_id": message_id,
            "timestamp": message_time,
            "sender": message["sender"],
            "text": message["text"],
        }

        # Agregar fecha y hora de inicio y fin de la conversación
        if i == 0:
            conversation_data["fecha_inicio"] = message_time
        if i == len(conversation["historial"]) - 1:
            conversation_data["fecha_fin"] = message_time

        # Se agrega el mensaje al array de mensajes de la conversación
        conversation_mensajes.append(message_data)

        start_time = current_time  # Actualizar start_time para el próximo mensaje

    # Se agrega el array de mensajes a la conversación
    conversation_data["messages"] = conversation_mensajes

    # Agregando support tickets
    if "support_ticket" in conversation:
        # Generar un support ticket para la conversación

        ticket_id = str(uuid.uuid4())
        ticket_created_at = conversation_data["fecha_fin"]
        estado = conversation["support_ticket"]["estado"]
        problema = conversation["support_ticket"]["problema"]
        descripcion = conversation["support_ticket"]["descripcion"]
        prioridad = conversation["support_ticket"]["prioridad"]

        support_ticket_random_hours = random.randint(
            0, 24 * 3
        )  # Hasta 3 días después de la creación
        support_ticket_fecha_cierre = start_time + timedelta(
            hours=support_ticket_random_hours
        )

        fecha_cierre = support_ticket_fecha_cierre if estado == "Cerrado" else None

        support_ticket = {
            "ticket_id": ticket_id,
            "user_id": conversation_data["user_id"],
            "prioridad": prioridad,
            "estado": estado,
            "problema": problema,
            "descripcion": descripcion,
            "fecha_creacion": conversation_data["fecha_fin"],
            "fecha_cierre": fecha_cierre,
        }

        conversation_data["support_ticket"] = support_ticket

    # Agregar feedback
    feedback_id = str(uuid.uuid4())
    if "feedback" in conversation:
        # Generar un feedback para la conversación
        rating = conversation["feedback"]["rating"]
        comentarios = conversation["feedback"]["comentarios"]
    else:
        rating = random.randint(1, 5)
        comentarios = random.choice(
            [
                "Excelente atención.",
                "Resolvieron mi problema rápidamente.",
                "El servicio podría mejorar.",
                "No quedé satisfecho con la respuesta.",
                "Muy buen soporte técnico.",
                "La conversación fue clara y útil.",
                "Me gustaría una atención más rápida.",
                "El agente fue muy amable.",
                "No entendí la solución propuesta.",
                "Todo perfecto, gracias.",
            ]
        )

    fecha_feedback = conversation_data["fecha_fin"]

    feedback = {
        "feedback_id": feedback_id,
        "session_id": session_id,
        "user_id": conversation_data["user_id"],
        "rating": rating,
        "comentarios": comentarios,
        "fecha_feedback": fecha_feedback,
    }

    conversation_data["feedback"] = feedback
    conversation_data["satisfaccion"] = (
        conversation["satisfaccion"] if "satisfaccion" in conversation else rating
    )

    # Agregar visita técnica
    if "visita_tecnica" in conversation:
        visita_id = str(uuid.uuid4())
        tecnico = conversation["visita_tecnica"]["tecnico"]
        fecha_visita = conversation["visita_tecnica"]["fecha_visita"]

        visita_tecnica_estado_choices = [
            # "Programada",
            # "En Curso",
            "Completada",
            "Cancelada",
            "Cerrada",
        ]
        visita_tecnica_estado = random.choice(visita_tecnica_estado_choices)

        visita_tecnica = {
            "visit_id": visita_id,
            "ticket_id": ticket_id if "support_ticket" in conversation else None,
            "user_id": conversation_data["user_id"],
            "tecnico": tecnico,
            "fecha_visita": fecha_visita,
            "estado": visita_tecnica_estado,
        }

        conversation_data["visita_tecnica"] = visita_tecnica

    conversations_data.append(conversation_data)

print(conversations_data[10])

Usuarios cargados desde PostgreSQL:
[RealDictRow([('user_id', 'ed3b8fb4-1999-4f05-b654-5b7df2aeef82'), ('nombre', 'Julieta Riba Borrego'), ('email', 'juan-luis71@example.net'), ('telefono', '+34732 823 807'), ('direccion', 'Paseo Amaro Duque 871\nAlmería, 46256'), ('fecha_registro', datetime.datetime(2024, 12, 29, 0, 25, 14))]), RealDictRow([('user_id', 'bb81a4eb-c7ed-4411-93a7-01f250c6a812'), ('nombre', 'Angelita de Salinas'), ('email', 'ariascristobal@example.org'), ('telefono', '+34810454551'), ('direccion', 'Vial de Fidela Mármol 93 Puerta 2 \nÁlava, 44651'), ('fecha_registro', datetime.datetime(2025, 2, 22, 0, 25, 14))]), RealDictRow([('user_id', 'd61e9531-e34d-4aa9-beeb-ab59f84c093e'), ('nombre', 'Custodia Pedro Batlle'), ('email', 'eliasapolonia@example.net'), ('telefono', '+34844901334'), ('direccion', 'Pasadizo de Eugenio Doménech 82 Puerta 1 \nLeón, 37097'), ('fecha_registro', datetime.datetime(2025, 2, 20, 0, 25, 14))]), RealDictRow([('user_id', 'deafc729-f17e-4864-a742-db55

In [28]:
messages = []
feedbacks = []
technical_visits = []
support_tickets = []
conversation_records = []
for conv in conversations_data:
    messages_tuples = [
        (
            msg["session_id"],
            msg["message_id"],
            msg["timestamp"],
            msg["sender"],
            msg["text"],
        )
        for msg in conv["messages"]
    ]
    messages.extend(messages_tuples)

    
    feedbacks_tuples = [
        (
            conv["feedback"]["feedback_id"],
            conv["feedback"]["session_id"],
            conv["feedback"]["user_id"],
            conv["feedback"]["rating"],
            conv["feedback"]["comentarios"],
            conv["feedback"]["fecha_feedback"],
        )
    ]
    feedbacks.extend(feedbacks_tuples)

    if "visita_tecnica" in conv:
        visita = conv["visita_tecnica"]
        technical_visits.append(
            (
                visita["visit_id"],
                visita["ticket_id"],
                visita["user_id"],
                visita["tecnico"],
                visita["fecha_visita"],
                visita["estado"],
            )
        )
    if "support_ticket" in conv:
        support_tickets.append(
            (
                conv["support_ticket"]["ticket_id"],
                conv["support_ticket"]["user_id"],
                conv["support_ticket"]["prioridad"],
                conv["support_ticket"]["estado"],
                conv["support_ticket"]["problema"],
                conv["support_ticket"]["descripcion"],
                conv["support_ticket"]["fecha_creacion"],
                conv["support_ticket"]["fecha_cierre"],
            )
        )
    conversation_records.append(
        (
            conv["session_id"],
            conv["user_id"],
            conv["fecha_inicio"],
            conv["fecha_fin"],
            conv["satisfaccion"],
        )
    )

In [29]:
# Insertando facturas
conversations_data_sql_query = "INSERT INTO conversations (session_id, user_id, fecha_inicio, fecha_fin, satisfaccion) VALUES (%s, %s, %s, %s, %s)"
print(conversation_records[:10])
insert_multiple_records(conversation_records, conversations_data_sql_query)

[('ad251405-c720-4766-89f9-04d38477ea11', '06a1d5d8-8edf-47d1-bfbc-d9e3afe2dbc5', '2025-06-11 00:53:02', '2025-06-11 01:06:02', 5), ('814a0298-7f79-49ef-abbe-9c1c94fe48b8', '3df25cff-7222-4b5a-867b-2ebf7da2353c', '2025-05-04 00:46:02', '2025-05-04 01:13:02', 2), ('1bc5ee82-9ed3-466b-a8d1-533ac24a4855', '80fcfef5-e766-42b8-bf5f-f009b079a2ba', '2025-07-19 00:39:02', '2025-07-19 01:39:02', 4), ('a3d229f4-dcad-4c9d-ac88-9a7e981e6f9a', 'deafc729-f17e-4864-a742-db5593c57a47', '2025-06-09 00:43:02', '2025-06-09 01:19:02', 1), ('4553b668-692a-4e87-ae77-14b231090afa', '843c7837-39f8-4fe7-9c8c-b07fc353009c', '2025-07-29 00:52:02', '2025-07-29 01:33:02', 5), ('459a745d-3dbf-4384-b303-4004ee3b41c0', 'e3af4890-5a12-4263-b28b-e7d3a6071f00', '2025-07-30 00:47:02', '2025-07-30 01:45:02', 5), ('92785e6d-ef65-4284-89b8-db6f6b3fd6a7', '4fe065d0-03cd-4675-84bd-ce0b1aee268a', '2025-07-06 00:51:02', '2025-07-06 01:28:02', 2), ('a27106a1-25cc-4441-bd92-d0efa4a9396a', '56cb92c7-4259-454f-990b-66060e70bc94', '

In [30]:
messages_sql_query = "INSERT INTO messages (session_id, message_id, timestamp, sender, text) VALUES (%s, %s, %s, %s, %s)"
insert_multiple_records(messages, messages_sql_query)

84 registros insertados correctamente.
Conexión a PostgreSQL cerrada.


In [31]:
feedbacks_sql_query = "INSERT INTO feedback (feedback_id, session_id, user_id, rating, comentarios, fecha_feedback) VALUES (%s, %s, %s, %s, %s, %s)"
insert_multiple_records(feedbacks, feedbacks_sql_query)

15 registros insertados correctamente.
Conexión a PostgreSQL cerrada.


In [33]:
support_tickets_sql_query = "INSERT INTO support_tickets (ticket_id, user_id, prioridad, estado, problema, descripcion, fecha_creacion, fecha_cierre) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
insert_multiple_records(support_tickets, support_tickets_sql_query)

7 registros insertados correctamente.
Conexión a PostgreSQL cerrada.


In [37]:
technical_visits_sql_query = "INSERT INTO technical_visits (visit_id, ticket_id, user_id, tecnico, fecha_visita, estado) VALUES (%s, %s, %s, %s, %s, %s)"
insert_multiple_records(technical_visits, technical_visits_sql_query)
print(technical_visits[:1])

3 registros insertados correctamente.
Conexión a PostgreSQL cerrada.
[('75fe8024-97a8-49ee-b4d7-78c95d334729', '2758f599-a476-4c29-8998-a3674aff4e06', '44fc9f3b-ce61-4dba-88dc-b21bd95e98a4', 'Carlos Pineda', '2025-10-08', 'Completada')]
